In [ ]:
from pyomo_orm.core.database import Session, bind_engine
from pyomo_orm.core.utils import as_dataframe

from models import *

In [ ]:
bind_engine('sqlite:///diet.sqlite')

In [ ]:
from pyomo.environ import AbstractModel, Param, PositiveReals, NonNegativeIntegers, Objective, Constraint, NonNegativeReals

diet = AbstractModel()
infinity = float('inf')

In [ ]:
# Sets
diet.food = Food.create_set()
diet.nutrients = Nutrient.create_set()

In [ ]:
# index params
diet.cost = Food.create_param(diet.food, from_attr='cost', indexed_by='id')
diet.volume_per_serving = Food.create_param(diet.food, from_attr='volume_per_serving', indexed_by='id')
diet.nutrient_lower_bound = Nutrient.create_param(diet.nutrients, from_attr='lower_bound', indexed_by='id', within=NonNegativeReals, default=0.0)
diet.nutrient_upper_bound = Nutrient.create_param(diet.nutrients, from_attr='upper_bound', indexed_by='id', within=NonNegativeReals, default=infinity)

In [ ]:
# multiindex params
diet.nutrient_amount = FoodNutrientAmount.create_param(diet.food, diet.nutrients, from_attr='amount', indexed_by=('food_id', 'nutrient_id'))

In [ ]:
diet.volume_max = Param(within=PositiveReals)

In [ ]:
# variable
diet.amount_in_diet = Food.create_var(diet, from_attr='amount_in_diet', indexed_by='id', within=NonNegativeIntegers)

In [ ]:
# constraints
def cost_rule(m):
    return (sum(m.cost[i] * m.amount_in_diet[i] for i in m.food))
diet.objective = Objective(rule=cost_rule)

def nutrient_rule_lower(m, j):
    value = sum(m.nutrient_amount[i, j] * m.amount_in_diet[i] for i in m.food)
    return m.nutrient_lower_bound[j] <= value

def nutrient_rule_upper(m,j):
    value = sum(m.nutrient_amount[i, j] * m.amount_in_diet[i] for i in m.food)
    return value <= m.nutrient_upper_bound[j]
diet.nutrient_constraint_lower = Constraint(diet.nutrients, rule=nutrient_rule_lower)
diet.nutrient_constraint_upper = Constraint(diet.nutrients, rule=nutrient_rule_upper)

def volume_rule(m):
    return sum(m.volume_per_serving[i] * m.amount_in_diet[i] for i in m.food) <= m.volume_max
diet.volume_constraint = Constraint(rule=volume_rule)

In [ ]:
from pyomo_orm.core.utils import create_data_dict

di = create_data_dict(diet, 'diet')
di['diet']['volume_max'] = {None: 75.0}

In [ ]:
di

In [ ]:
from pyomo.environ import SolverFactory

instance = diet.create_instance(data=di, namespace='diet')
solver = SolverFactory('cbc')
results = solver.solve(instance, tee=True)

In [ ]:
as_dataframe(Food.query())

In [ ]:
instance.amount_in_diet.extract_values()

In [ ]:
import pandas as pd
pd.Series(instance.amount_in_diet.extract_values())

In [ ]:
instance.write()

In [ ]:
Food.query().filter(Food.id.in_(list(di['diet']['cost'].keys()))).all()

In [ ]:
list(instance.food)

In [ ]:
list(instance.cost)

In [ ]:
Food.__name__